<a href="https://colab.research.google.com/github/AndrzejOlejniczak/PORTFOLIO/blob/main/Graph_Neural_Networks_with_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Neural Networks in Tensorflow
This notebook follows this TensorFlow Exercise: https://www.youtube.com/watch?v=8owQBFAHw7E&t=1071s

## 0. Prepare the environment

In [ ]:
# Install necessary modules
!pip install spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import necessary modules
import numpy as np
import tensorflow as tf
import spektral

## 1. Download and prepare the data

In [ ]:
# Download adjacency matrix, feature matrix, labels and masks for dicriminating train, validation and test subsets
cora_dataset= spektral.datasets.citation.Citation(name='cora')

# Convert features and adjancency marix from sparse to dense representation
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y

# Convert adjacency matrix to dense representation
adj = adj.todense() + np.eye(adj.shape[0])

# Convert features and adjacency matrix to tf.float32
features = features.astype('float32')
adj = adj.astype('float32')


In [ ]:
# Check the data
print(features.shape)
print(adj.shape)
print(labels.shape)

# Check the masks
print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


## 2. Build the model

### 2.1. Define masekd loss and masked accuracy functions

In [ ]:
# Define a function that computes softmax cross entropy on masked data
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
  mask = tf.cast(mask, dtype= tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)

In [ ]:
# Define function that computes accuracy on masekd data
def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

### 2.2. Define graph neural network layer
Graph neuralnetwork layer looks at feature matrix, adjacency matrix, some transformation to apply to the nodes and activation function

In [ ]:
# Define graph neural network layer
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)

### 2.3. Build a model without tramsformation

In [ ]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7) # Cora dataset has 7 classes

  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

  best_accuracy = 0.0
  for ep in range(epochs + 1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads, variables))

    logits = cora_gnn(fts, adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    test_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      print('Epoch: ', ep, '| Training loss: ', loss.numpy(), '| Val accuracy: ', val_accuracy.numpy(), '| Test accuracy: ', test_accuracy.numpy())

In [ ]:
# Train on the data
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch:  0 | Training loss:  5.1528645 | Val accuracy:  0.116 | Test accuracy:  0.144
Epoch:  1 | Training loss:  8.18531 | Val accuracy:  0.332 | Test accuracy:  0.377
Epoch:  2 | Training loss:  3.6909916 | Val accuracy:  0.378 | Test accuracy:  0.405
Epoch:  3 | Training loss:  2.9333885 | Val accuracy:  0.48599997 | Test accuracy:  0.529
Epoch:  4 | Training loss:  1.9079802 | Val accuracy:  0.534 | Test accuracy:  0.57199997
Epoch:  5 | Training loss:  0.89222723 | Val accuracy:  0.626 | Test accuracy:  0.655
Epoch:  6 | Training loss:  0.5559627 | Val accuracy:  0.63600004 | Test accuracy:  0.63699996
Epoch:  7 | Training loss:  0.5389886 | Val accuracy:  0.642 | Test accuracy:  0.64599997
Epoch:  9 | Training loss:  0.41717646 | Val accuracy:  0.662 | Test accuracy:  0.672
Epoch:  10 | Training loss:  0.3469951 | Val accuracy:  0.67399997 | Test accuracy:  0.701
Epoch:  11 | Training loss:  0.2948745 | Val accuracy:  0.684 | Test accuracy:  0.718
Epoch:  12 | Training loss:  0.26

_Experiment_: Below is to check how the structure of the graph affects information transformations. How does removing edges affect the training?

In [ ]:
# Train on the data using only identity matrix insead of adjacency matrix
train_cora(features, tf.eye(adj.shape[0]), gnn, 32, 200, 0.01)

Epoch:  0 | Training loss:  1.9456255 | Val accuracy:  0.22999999 | Test accuracy:  0.24499999
Epoch:  1 | Training loss:  1.6622331 | Val accuracy:  0.31 | Test accuracy:  0.339
Epoch:  2 | Training loss:  1.4103558 | Val accuracy:  0.36999997 | Test accuracy:  0.383
Epoch:  3 | Training loss:  1.1556315 | Val accuracy:  0.41400003 | Test accuracy:  0.41199994
Epoch:  4 | Training loss:  0.9103684 | Val accuracy:  0.44599998 | Test accuracy:  0.43799996
Epoch:  5 | Training loss:  0.6932017 | Val accuracy:  0.464 | Test accuracy:  0.45499995
Epoch:  6 | Training loss:  0.514405 | Val accuracy:  0.484 | Test accuracy:  0.47199997
Epoch:  7 | Training loss:  0.37412757 | Val accuracy:  0.49199998 | Test accuracy:  0.49099997
Epoch:  8 | Training loss:  0.27071223 | Val accuracy:  0.52 | Test accuracy:  0.509
Epoch:  9 | Training loss:  0.19766565 | Val accuracy:  0.54599994 | Test accuracy:  0.51299995
Epoch:  10 | Training loss:  0.14696161 | Val accuracy:  0.55799997 | Test accuracy: 

## 2.4. Build a model dividing node adjacency with the node degree

In [ ]:
# Compute graph degree vector
deg = tf.reduce_sum(adj, axis=-1)
deg.shape

TensorShape([2708])

In [ ]:
# Train
train_cora(features, adj / deg, gnn, 32, 200, 0.01 )

Epoch:  0 | Training loss:  1.9413778 | Val accuracy:  0.466 | Test accuracy:  0.501
Epoch:  1 | Training loss:  1.7309524 | Val accuracy:  0.584 | Test accuracy:  0.58699995
Epoch:  2 | Training loss:  1.4966334 | Val accuracy:  0.652 | Test accuracy:  0.66499996
Epoch:  3 | Training loss:  1.2567844 | Val accuracy:  0.70000005 | Test accuracy:  0.71699995
Epoch:  4 | Training loss:  1.0330662 | Val accuracy:  0.74799997 | Test accuracy:  0.76800007
Epoch:  6 | Training loss:  0.67012167 | Val accuracy:  0.752 | Test accuracy:  0.794
Epoch:  7 | Training loss:  0.5334632 | Val accuracy:  0.762 | Test accuracy:  0.79699993
Epoch:  8 | Training loss:  0.42164192 | Val accuracy:  0.782 | Test accuracy:  0.79699993


## 2.5. Build a model using symmetrical degree

In [ ]:
# Compute normalized degree
norm_deg = tf.linalg.diag(1.0 / tf.sqrt(deg))

# Compute normalized adjacency
norm_adj = tf.matmul(norm_deg, tf.matmul(adj, norm_deg))

In [ ]:
# Train
train_cora(features, norm_adj, gnn, 32, 200, 0.01)

Epoch:  0 | Training loss:  1.9467878 | Val accuracy:  0.53 | Test accuracy:  0.564
Epoch:  1 | Training loss:  1.7959969 | Val accuracy:  0.626 | Test accuracy:  0.667
Epoch:  2 | Training loss:  1.615622 | Val accuracy:  0.658 | Test accuracy:  0.709
Epoch:  3 | Training loss:  1.4091548 | Val accuracy:  0.68 | Test accuracy:  0.72199994
Epoch:  4 | Training loss:  1.2008429 | Val accuracy:  0.722 | Test accuracy:  0.746
Epoch:  5 | Training loss:  1.0074307 | Val accuracy:  0.75600004 | Test accuracy:  0.7779999
Epoch:  6 | Training loss:  0.8323209 | Val accuracy:  0.76199996 | Test accuracy:  0.78699994
Epoch:  7 | Training loss:  0.6759271 | Val accuracy:  0.77800006 | Test accuracy:  0.79699993
Epoch:  11 | Training loss:  0.2630596 | Val accuracy:  0.786 | Test accuracy:  0.78900003
Epoch:  12 | Training loss:  0.20421101 | Val accuracy:  0.79 | Test accuracy:  0.79499996
Epoch:  13 | Training loss:  0.15813668 | Val accuracy:  0.79200006 | Test accuracy:  0.798
Epoch:  14 | Tr